# IUM-zespół 5 wariant 02
Marcin Jarczewski
Miłosz Mizak

# Wstęp

W ramach drugiego etapu projektu stworzyliśmy:
- model bazowy
- model zaawansowany (docelowy)
- API służące do interakcji z modelami

## Klasy

Oba modele korzystają z klas reprezentujących piosenkę oraz składankę. Klasa `Track` jest bardzo prosta i zawiera tylko trzy pola: `track_id`, `popularity` i `group`. Klasa `Playlist` jest już nieco bardziej skomplikowana. Zawiera ona funkcje, które aktualizują informacje na temat składanki za każdym razem, gdy dodawana jest nowa piosenka — jej wielkość, statystyki na temat popularności piosenek w składance, a także łączną długość wszystkich piosenek.

## Model bazowy

Model bazowy działa w bardzo prosty sposób: tworzy składanki z tych gatunków muzycznych, które posiadają więcej niż X piosenek, gdzie X jest parametrem modelu. Domyślnie jest to ustawione na wartość 500 piosenek. Po wygenerowaniu składanek z najpopularniejszymi piosenkami z danych gatunków model wybiera te składanki, które mają największą średnią popularność. Ilość tych składanek także jest parametrem modelu.

## Model zaawansowany

Główną funkcją modelu zaawansowanego jest grupowanie piosenek. Użyliśmy do tego gotowego modelu KMeans, dostępnego w bibliotece scikit-learn. Model działa w następujący sposób:
- model przyporządkowuje każdą piosenkę ze zbioru danych do określonej grupy. Testowaliśmy różne ilości grup i przy obecnym rozmiarze zbioru danych 10 grup wydaje się rozsądnym wyborem. Większa ich ilość generowała patologie — przy 30 grupach piosenki o dokładnie takich samych parametrach były przyporządkowywane do zupełnie innych grup.
- piosenki zostają posortowane ze względu na grupy, a następnie z danych wybierane są te piosenki, które przekraczają określony próg popularności (parametr `popularity`) - jako domyślna wartość jest to 95. 
- popularne piosenki znajdują się teraz w odpowiednich grupach, więc inne piosenki powinny być do nich podobne — wobec tego z grup w którch znajdują się te piosenki, losujemy daną ilość piosenek.
- wylosowane piosenki dodajemy do składanek, które potem są wszystkie razem zwracane jako wynik działania modelu.

Różnice w działaniu modeli widać na pierwszy rzut oka: średnia popularność w modelu zaawansowanym jest niższa (zazwyczaj na poziomie ok. 60) niż w modelu bazowym. Nie oznacza to jednak, że model zaawansowany nie działa. W składance nie muszą znajdować się tylko same najpopularniejsze piosenki, aby słuchacze byli nią zainteresowani. Według nas, jeżeli mniej popularne piosenki będą podobne do tych popularnych, to ich wybór do składanki nie wpłynie negatywnie na doświadczenie użytkowników. Dodatkowo generowanie częściowo losowych playlist zwiększa ich różnorodność. Opieranie się tylko na popularności piosenek będzie generowało bardzo podobne składanki, gdyż wahania popularności w czasie nie są wystarczająco duże.


## Serwis

Do uruchamiania modeli zostało oddelegowane API. API posiada dwa endpointy jeden z nich: `/predict/model/{model_type}`, gdzie jako `model_type` można użyć zarówno modelu bazowego, jak i zaawansowanego. Liczba generowanych składanek zawsze wynosi 5, natomiast istnieje możliwość sprecyzowania ilości piosenek w składance poprzez modyfikację pliku `config.py`.

Drugi endpoint służy do przeprowadzenia testów A/B: `/next-playlist/{user_id}`, na podstawie reszty z dzielenia przez dwa jest przypisywany model, który zostanie użyty do wygenerowania składanki.

Aby przetestować działanie mikroserwisu, należy najpierw pobrać wszystkie niezbędne biblioteki używając komendy `pip -r requirements.txt`, a następnie uruchomić `cd src/infra && python service.py`. Na adresie `0.0.0.0:8000` zostanie uruchomiony serwis.

![Api](./figures/api.png)

Dodatkowo można uruchomić nasz serwis przy wykorzystaniu Dockera, korzystając z poleceń:
1. `docker build -t ium-service-image .`
2. `docker run -it --rm --name ium-service -p 8000:8000 ium-service-image`

Analoginie serwis będzie dostepny pod adresem `0.0.0.0:8000`.

## Obserwacje

Specyfika zadania sprawia, że przetestowanie modeli offline jest trudnym i kosztownym zadaniem. Implementacja modeli działa — modele generują składanki o określonej ilości piosenek. Ale żeby zbadać, czy te składanki faktycznie przełożą się na większą retencję wśród użytkowników, konieczne będą testy online. Dane, które obecnie posiadamy nie uwzględniają zmian w zachowaniu użytkowników po wprowadzeniu składanek generowanych przez model.

Czysto teoretycznie byłaby możliwość przeprowadzenia testów offline. Wymagałoby to jednak stworzenia drugiego modelu, który przewidywałby zachowanie użytkowników. Wiąże się z tym jednak parę problemów:
- należałoby założyć, że model użytkownika jest prawidłowy i dobrze przewiduje jego zachowanie
- stworzenie takiego modelu wykracza poza nasze zadanie
- trudno ocenić, czy z danymi, które obecnie posiadamy, byłaby możliwość przewidzenia zachowania użytkownika w nowym środowisku, z nowymi składankami

Wobec powyższych problemów nie będziemy realizowali takiego wariantu.